# Use case examples
This notebook demonstrates different use-cases for QC Lab

## Running a default simulation

In [2]:
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

100%|██████████| 10001/10001 [00:03<00:00, 2763.90it/s]


## Changing the model ingredients

In [6]:
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np
from qc_lab.ingredients import harmonic_oscillator_wigner_init_classical

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
print(sim.model.ingredients[5])
# change the init_classical ingredient from boltzmann to wigner.
sim.model.ingredients[5] = ('init_classical', harmonic_oscillator_wigner_init_classical)
print(sim.model.ingredients[5])
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

('init_classical', <function harmonic_oscillator_boltzmann_init_classical at 0x70022c872440>)
('init_classical', <function harmonic_oscillator_wigner_init_classical at 0x70022c8724d0>)


100%|██████████| 10001/10001 [00:03<00:00, 2706.01it/s]


## Changing algorithm tasks

In [9]:
from qc_lab import Simulation
from qc_lab.dynamics import serial_driver, parallel_driver_multiprocessing
from qc_lab.models import SpinBoson
from qc_lab.algorithms import MeanField
import numpy as np

sim_settings = {"num_trajs":100,"batch_size":25}

sim = Simulation(sim_settings)
sim.model = SpinBoson()
sim.algorithm = MeanField()
sim.state.wf_db = np.array([1.0, 0.0], dtype=complex)
print(sim.algorithm.output_recipe)
def my_task(algorithm, sim, parameters, state):
    # we can just calculate something simple like the response function
    if sim.t_ind == 0:
        state.wf_db_0 = np.copy(state.wf_db)
    state.response = np.sum(np.conj(state.wf_db_0) * state.wf_db, axis=-1)
    return parameters, state
sim.algorithm.output_recipe.append(my_task)
sim.algorithm.output_variables.append('response')
print(sim.algorithm.output_recipe)
data = serial_driver(sim)
data = parallel_driver_multiprocessing(sim)

[<function update_t at 0x70022c89c790>, <function update_dm_db_mf at 0x70022c89c820>, <function MeanField._update_quantum_energy at 0x70022c89d7e0>, <function MeanField._update_classical_energy at 0x70022c89d870>]
[<function update_t at 0x70022c89c790>, <function update_dm_db_mf at 0x70022c89c820>, <function MeanField._update_quantum_energy at 0x70022c89d7e0>, <function MeanField._update_classical_energy at 0x70022c89d870>, <function my_task at 0x70024017e8c0>]


100%|██████████| 10001/10001 [00:03<00:00, 2759.69it/s]
